In [28]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import datetime
import re

ticker = '2330'
url = f'https://udn.com/search/word/2/{ticker}'  
response = requests.get(url)
titles = []

current_datetime = datetime.datetime.now()
current_datetime = pd.to_datetime(current_datetime.strftime('%Y-%m-%d')) 

if response.status_code == 200:

    soup = BeautifulSoup(response.text, 'html.parser')

    links = soup.find_all(class_='story-list__news')
    
    for tag in links:
        
        title = tag.find('h2')
        if title:
            out = title.text
            titles.append(out)
        
        date = tag.find('time').text
        date_time = pd.to_datetime(date)
        
        if date_time < current_datetime:
            break
        
else:
    print("access denied")
    
titles = list(set(titles))

titles = [re.sub('[\n]+', '', i) for i in titles]
titles

['台積擴建破局 考驗處理智慧',
 '台積電進駐計畫喊卡 龍潭房市面臨失望性賣壓',
 '長焦鏡頭大漲3.8倍…i15拆解 台鏈三贏家出列 ',
 '全民權證／台積、鴻海 認購放閃',
 '經部：半導體先進製程 根留台灣',
 '法說會前夕喊卡 台積電龍潭廠不蓋了',
 'Nvidia下挫收黑！台股開盤跌逾30點失守半年線 台積電跌2元開549元',
 '台積電進駐龍科三期喊卡 高雄、台中、台南都在招手',
 '台積電法說能讓台股上萬七？分析師曝「最大變數」必影響股價']

In [26]:
import tejapi

api_key = 'api key'
tejapi.ApiConfig.api_base="http://api.tej.com.tw"
tejapi.ApiConfig.api_key = api_key
tejapi.ApiConfig.ignoretz = True

gte = pd.to_datetime(current_datetime) - datetime.timedelta(days=20)

stock = tejapi.get('TWN/APRCD',
                   mdate = {'gte':gte, 'lte':current_datetime},
                   coid = ticker,
                   chinese_column_name = True

)

stock

,證券代碼,年月日,開盤價(元),最高價(元),最低價(元),收盤價(元),成交量(千股),成交值(千元),報酬率％,週轉率％,...,現金股利率,股價漲跌(元),高低價差%,次日開盤參考價,次日漲停價,次日跌停價,注意股票(A),處置股票(D),全額交割(Y),市場別
None,,,,,,,,,,,,,,,,,,,,,
0,2330,2023-09-28,523.0,525.0,520.0,523.0,28350.350,1.482625e+07,0.1916,0.1093,...,16.3475,1.0,0.9579,523.0,575.0,471.0,,,,TSE
1,2330,2023-10-02,530.0,534.0,528.0,533.0,26891.996,1.430068e+07,1.9120,0.1037,...,16.0408,10.0,1.1472,533.0,586.0,480.0,,,,TSE
2,2330,2023-10-03,528.0,533.0,528.0,529.0,17601.440,9.335110e+06,-0.7505,0.0679,...,16.1621,-4.0,0.9381,529.0,581.0,476.5,,,,TSE
3,2330,2023-10-04,521.0,523.0,519.0,520.0,29808.729,1.551974e+07,-1.7013,0.1149,...,16.4418,-9.0,0.7561,520.0,572.0,468.0,,,,TSE
4,2330,2023-10-05,523.0,529.0,523.0,528.0,25749.184,1.356394e+07,1.5385,0.0993,...,16.1927,8.0,1.1538,528.0,580.0,475.5,,,,TSE
5,2330,2023-10-06,530.0,533.0,529.0,532.0,16160.314,8.587684e+06,0.7576,0.0623,...,16.0709,4.0,0.7576,532.0,585.0,479.0,,,,TSE
6,2330,2023-10-11,542.0,544.0,540.0,544.0,61670.562,3.347797e+07,2.2556,0.2378,...,15.7164,12.0,0.7519,544.0,598.0,490.0,,,,TSE
7,2330,2023-10-12,545.0,550.0,544.0,550.0,36350.241,1.991679e+07,1.1029,0.1402,...,15.5450,6.0,1.1029,550.0,605.0,495.0,,,,TSE
8,2330,2023-10-13,550.0,554.0,548.0,553.0,34704.924,1.912810e+07,0.5455,0.1338,...,15.4606,3.0,1.0909,553.0,608.0,498.0,,,,TSE


In [12]:
# pip install --upgrade langchain

In [15]:
import os
import openai
os.environ['OPENAI_API_KEY'] = 'api key'
openai.api_key = os.environ['OPENAI_API_KEY']

In [8]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferWindowMemory

In [30]:
prompt = str(titles)+'請幫我用以上標題用中文分析市場情緒，總體而言正面多給我’正面‘這兩個字，負面多給我‘負面’這兩個字就好'分析報告

In [31]:
llm_model = 'gpt-3.5-turbo-0301'
llm = ChatOpenAI(temperature=0.0, model=llm_model)
memory = ConversationBufferWindowMemory(k=1)
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)
sentiment = conversation.predict(input=prompt)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: ['台積擴建破局 考驗處理智慧', '台積電進駐計畫喊卡 龍潭房市面臨失望性賣壓', '長焦鏡頭大漲3.8倍…i15拆解 台鏈三贏家出列 ', '全民權證／台積、鴻海 認購放閃', '經部：半導體先進製程 根留台灣', '法說會前夕喊卡 台積電龍潭廠不蓋了', 'Nvidia下挫收黑！台股開盤跌逾30點失守半年線 台積電跌2元開549元', '台積電進駐龍科三期喊卡 高雄、台中、台南都在招手', '台積電法說能讓台股上萬七？分析師曝「最大變數」必影響股價']請幫我用以上標題用中文分析市場情緒，總體而言正面多給我’正面‘這兩個字，負面多給我‘負面’這兩個字就好
AI:

> Finished chain.


'根據以上標題分析，市場情緒整體而言較為負面。其中，台積電進駐計畫喊卡、龍潭房市面臨失望性賣壓、台積電龍潭廠不蓋了等標題都帶有負面情緒。此外，Nvidia下挫收黑、台股開盤跌逾30點失守半年線等標題也顯示出市場整體情緒較為悲觀。然而，全民權證／台積、鴻海認購放閃、經部：半導體先進製程根留台灣等標題則帶有一定的正面情緒。總體而言，市場情緒較為負面。'